In [10]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy
import itertools
import pickle as pk

This script focuses on performing differential gene expression analysis on the data corresponding to different respiratory diseases. The genes that are not satistically significant are filtered out for the next step of analysis and to reduce the size of the network 

In [5]:
path = os.getcwd()+'/normalized data/'
os.listdir(path)

['respiratory_data.csv',
 'covid_data.csv',
 'respiratory_metadata.csv',
 'covid_metadata.csv']

## DEGs Function

In [54]:
def compute_dynamic_dge(mapped_omics, sample_file, test_type='t', log=True, combo_func = 'gmean', transform_zero = False):
    """Performs dynamic differential gene expression analysis.
    
    Given a mapped omics DataFrame and a sample file with columns `'sample'`, `'condition'` and `'time'`, performs a differential
    expression analysis between all pairs of conditions at all available time points. Uses the Mann-Whitney U-test to compute the
    pvalues of differential expression. Note that the sample names in `'sample'` column must match the column names of the mapped omics.
    
    Args:
        mapped_omics (:obj:`pd.DataFrame`): containing RMA normalized gene expressed for gene(s) (see output of ``parse_transcriptomics()``)
        sample_file (:obj:`str`): name of a CSV file which contains experimental data; must contain columns with headers `'sample'`, `'condition'` and `'time'`
        log (:obj:`bool`): whether to log the expression; default = `True`
        combo_func (:obj:`str`): the function used to combine values of different samples of same condition. Possible strings can be 'gmean' for gemoetric mean, 'mean' and 'median' for arthimetic mean and median respectively.
        transform_zero (:obj:`bool`): whether to apply log(x+k) transform to values of genes with the number of zeros greater than a threshold
    Returns:
        A :obj:`dict` whose keys correspond to a :obj:`tuple` of ``(condition_1, condition_2, time_point)``, and value corresponds to a 
        :obj:`pd.DataFrame` containing the output of differential gene expression analysis from ``condition_1`` to ``condition_2`` at given ``time_point``
    See also:
        * Mann-Whitney U-test: https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test
    """
    import scipy
    from scipy.stats import ttest_ind
    from scipy.stats import mannwhitneyu
    from scipy.stats import gmean
    from math import floor
    from operator import truediv
    from statsmodels.stats.multitest import multipletests
    import itertools
    import warnings
    
    samples = pd.read_csv(sample_file, sep=',', header=0, index_col=0, dtype=str)
    groups = np.unique(samples['condition'])
    timept = np.unique(samples['time'])
    print('Found condition groups', groups, 'at time points', timept)
    out = dict()
    for (i,j) in itertools.product(groups, groups):
        if i==j: continue
        else:
            for t in timept:
                geneexp_i = mapped_omics[samples[(samples['condition']==i) & (samples['time']==t)].index].astype('float64').values
                geneexp_j = mapped_omics[samples[(samples['condition']==j) & (samples['time']==t)].index].astype('float64').values
                if geneexp_i.size>0 and geneexp_j.size>0:
                    geo_mean_i = []
                    geo_mean_j = []
                    if geneexp_i.shape[0] != geneexp_j.shape[0]:
                        print('unequal matrix size')
                        break
                    geneexp_ij = np.append(geneexp_i, geneexp_j, 1)
                    zero_transformed = [np.nan]*geneexp_i.shape[0]
                    for r in range(geneexp_i.shape[0]):
                        if np.count_nonzero(geneexp_i[r]) < floor(0.75*geneexp_i.shape[1]) or np.count_nonzero(geneexp_j[r]) < floor(0.75*geneexp_j.shape[1]):
                            if transform_zero:
                                #print(len(np.nonzero(geneexp_ij[r])), np.nonzero(geneexp_ij[r]))
                                zero_transformed[r] = 'yes'
                                if np.count_nonzero(geneexp_ij[r]) == 0:
                                    add_min = 1
                                else:
                                    add_min = np.min(geneexp_ij[r][np.nonzero(geneexp_ij[r])])/2
                                geneexp_i[r] += add_min
                                geneexp_j[r] += add_min
                            else:
                                geneexp_i[r] = np.array([[np.nan]*geneexp_i.shape[1]])
                                geneexp_j[r] = np.array([[np.nan]*geneexp_j.shape[1]])
                        
                        if combo_func == 'gmean':
                            geo_mean_i.append(gmean([x for x in geneexp_i[r] if x!=0]))
                            geo_mean_j.append(gmean([x for x in geneexp_j[r] if x!=0]))
                        if combo_func == 'mean':
                            geo_mean_i.append(np.mean(geneexp_i[r]))
                            geo_mean_j.append(np.mean(geneexp_j[r]))
                        if combo_func == 'median':
                            geo_mean_i.append(np.median(geneexp_i[r]))
                            geo_mean_j.append(np.median(geneexp_j[r]))
                            
                    fc_vals = list(map(truediv, geo_mean_j, geo_mean_i))
                
                    if log:
                        log2fcval = np.log2(fc_vals)
                    else:
                        log2fcval = np.array([-1/i if i<1 else i for i in fc_vals])
                    
                    stat = np.zeros(log2fcval.size)
                    pval = np.zeros(log2fcval.size)
                    prob = np.zeros(log2fcval.size)
                    for idx in range(log2fcval.size):
                        if test_type == 'u':
                            statistic, pvalue = mannwhitneyu(geneexp_i[idx,:], geneexp_j[idx,:])
                        else:
                            statistic, pvalue = ttest_ind(geneexp_i[idx,:], geneexp_j[idx,:], equal_var = False)
                        stat[idx] = statistic
                        pval[idx] = pvalue
                        if log2fcval[idx]>0: prob[idx] = 1-pval[idx]
                        else: prob[idx] = pval[idx]
                    out[(i,j,t)] = pd.DataFrame(np.vstack([mapped_omics['genes'], prob, log2fcval, pval, stat, zero_transformed]).transpose(), columns=['genes', 'probability', 'log2fc', 'pvalue', 'statistic','zero_transformed'])
                    
                if np.count_nonzero(out[(i,j,t)].log2fc.isnull()) >= floor(0.4*out[(i,j,t)].shape[0]):
                    warnings.warn('Log fold change is not defined for over 40% of genes for condition set {}.\nConsider transforming zero values'.format((i,j,t)))
    if len(out)==0: print('Warning, no appropriate condition groups found! Ensure at last 1 sample for every condition at every time point of interest.')
    return out


In [55]:
nemo=compute_dynamic_dge(mapped_omics, sample_file, test_type='t', log=True, combo_func = 'mean', transform_zero = False)

KeyError: 'time'

In [13]:
mapped_omics = pd.read_csv(path+'respiratory_data.csv')
sample_file = path + 'respiratory_metadata.csv'

In [39]:
def calculate_degs(mapped_omics, sample, combo_func = 'mean', test_type = 'student_t'):
    from scipy.stats import gmean
    from scipy.stats import ttest_ind
    from scipy.stats import mannwhitneyu
    from operator import truediv
    import itertools

    samples = pd.read_csv(sample_file, sep=',', header=0, index_col=0, dtype=str)
    groups = np.unique(samples['condition'])
    print('Found condition groups', groups)
    out = dict()
    for (i,j) in itertools.product(groups, groups):
        if i==j: continue
        else: 
            print('calculating degs for ({},{})'.format(i,j))
            # extracting gene expression values of all samples of a given condition
            geneexp_i = mapped_omics[samples[samples['condition']==i].index].astype('float64').values
            geneexp_j = mapped_omics[samples[samples['condition']==j].index].astype('float64').values
        
            geo_mean_i = []
            geo_mean_j = []
        
            # combining gene expression values of samples with the same condition
            if combo_func == 'gmean':
                if np.count_nonzero(geneexp_i) != geneexp_i.shape[0]*geneexp_i.shape[1] or \
                np.count_nonzero(geneexp_j) != geneexp_j.shape[0]*geneexp_j.shape[1]:
                    print('Genes with zero counts encountered. Cannot calculate geometric mean')
                    return 
                geo_mean_i = [gmean(geneexp_i[r]) for r in range(0, geneexp_i.shape[0])]
                geo_mean_j = [gmean(geneexp_j[r]) for r in range(0, geneexp_j.shape[0])]
            if combo_func == 'mean':
                geo_mean_i = [np.mean(geneexp_i[r]) for r in range(0, geneexp_i.shape[0])]
                geo_mean_j = [np.mean(geneexp_j[r]) for r in range(0, geneexp_j.shape[0])]
            if combo_func == 'median':
                geo_mean_i = [np.median(geneexp_i[r]) for r in range(0, geneexp_i.shape[0])]
                geo_mean_j = [np.median(geneexp_j[r]) for r in range(0, geneexp_j.shape[0])]
            
            # calculating log fold change
            fc_vals = list(map(truediv, geo_mean_j, geo_mean_i))
            log2fcval = np.log2(fc_vals)
        
            # performing hypothesis tests
            pval = np.zeros(len(fc_vals))
            test_statistic = np.zeros(len(fc_vals))
        
            for idx in range(len(fc_vals)):
                if test_type == 'student_t':
                    statistic, pvalue = ttest_ind(geneexp_i[idx,:], geneexp_j[idx,:], equal_var = False)
                if test_type == 'mann_u':
                    statistic, pvalue = mannwhitneyu(geneexp_i[idx,:], geneexp_j[idx,:])
                test_statistic[idx] = statistic
                pval[idx] = pvalue
        
            out[(i,j)] = pd.DataFrame({'genes':mapped_omics.genes, 'statistic':test_statistic, 'pvalue': pval})
    return out

In [40]:
temp = calculate_degs(mapped_omics, sample_file)

Found condition groups ['Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus'
 'Dengue' 'Influenza' 'Parainfluenza_RespiratorySyncytial' 'Pneumonia'
 'Rhinovirus' 'healthy_ctrl']
calculating degs for (Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus,Dengue)
calculating degs for (Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus,Influenza)
calculating degs for (Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus,Parainfluenza_RespiratorySyncytial)
calculating degs for (Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus,Pneumonia)
calculating degs for (Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus,Rhinovirus)
calculating degs for (Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus,healthy_ctrl)
calculating degs for (Dengue,Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus)
calculating degs for (Dengue,Influenza)
calculating degs for (Dengue,Parainfluenza_

In [51]:
np.count_nonzero(temp[('healthy_ctrl','Adenovirus_Cytomegalovirus_Ebstein-Barr virus_Herpes Simplex virus')].pvalue < 0.005)

1466

## GSE157240 - Respiratory Disease